In [1]:
#파일 불러오기
from google.colab import drive
drive.mount('/gdrive')
from scipy import io


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
#파일불러오기 2
test_32x32 = io.loadmat('/gdrive/My Drive/deeplearning/test_32x32.mat')

In [4]:

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


TensorFlow 2.x selected.


In [5]:
#x,y값 받아오기
x=np.array(test_32x32.get('X'))
y=np.array(test_32x32.get('y'))
x.shape,y.shape

((32, 32, 3, 26032), (26032, 1))

In [0]:
#x값 모양 변경
x=np.swapaxes(x,2,3)
x=np.swapaxes(x,1,2)
x=np.swapaxes(x,0,1)


In [0]:
#y값 변경
y=np.reshape(y,(-1,))

In [8]:
#확인
x.shape,y.shape

((26032, 32, 32, 3), (26032,))

In [9]:
print(x)

[[[[ 38 103  60]
   [ 39 104  61]
   [ 39 104  62]
   ...
   [ 41 102  61]
   [ 42 103  62]
   [ 39  97  57]]

  [[ 39 104  61]
   [ 39 104  61]
   [ 39 104  62]
   ...
   [ 41 102  61]
   [ 43 101  63]
   [ 39  97  57]]

  [[ 38 105  62]
   [ 37 104  61]
   [ 39 106  63]
   ...
   [ 43 101  63]
   [ 43 100  64]
   [ 39  97  59]]

  ...

  [[ 52 119  78]
   [ 53 120  79]
   [ 51 118  77]
   ...
   [ 44 118  69]
   [ 44 117  71]
   [ 41 114  69]]

  [[ 50 117  76]
   [ 51 118  77]
   [ 49 116  75]
   ...
   [ 44 117  71]
   [ 45 116  72]
   [ 42 113  69]]

  [[ 48 115  74]
   [ 48 115  74]
   [ 46 113  72]
   ...
   [ 43 116  71]
   [ 44 115  71]
   [ 42 113  71]]]


 [[[129 142 153]
   [127 143 152]
   [125 143 151]
   ...
   [121 133 153]
   [123 134 156]
   [123 135 157]]

  [[134 150 160]
   [133 149 158]
   [132 149 157]
   ...
   [127 138 157]
   [126 137 157]
   [125 137 158]]

  [[141 158 168]
   [140 157 166]
   [140 157 165]
   ...
   [135 147 163]
   [132 143 161]
   [130 141

In [10]:
#y의 라벨 변경(10->0)
y=np.where(y==10,0,y)
y

array([5, 2, 1, ..., 7, 6, 7], dtype=uint8)

In [11]:
x=x/255

x

array([[[[0.14901961, 0.40392157, 0.23529412],
         [0.15294118, 0.40784314, 0.23921569],
         [0.15294118, 0.40784314, 0.24313725],
         ...,
         [0.16078431, 0.4       , 0.23921569],
         [0.16470588, 0.40392157, 0.24313725],
         [0.15294118, 0.38039216, 0.22352941]],

        [[0.15294118, 0.40784314, 0.23921569],
         [0.15294118, 0.40784314, 0.23921569],
         [0.15294118, 0.40784314, 0.24313725],
         ...,
         [0.16078431, 0.4       , 0.23921569],
         [0.16862745, 0.39607843, 0.24705882],
         [0.15294118, 0.38039216, 0.22352941]],

        [[0.14901961, 0.41176471, 0.24313725],
         [0.14509804, 0.40784314, 0.23921569],
         [0.15294118, 0.41568627, 0.24705882],
         ...,
         [0.16862745, 0.39607843, 0.24705882],
         [0.16862745, 0.39215686, 0.25098039],
         [0.15294118, 0.38039216, 0.23137255]],

        ...,

        [[0.20392157, 0.46666667, 0.30588235],
         [0.20784314, 0.47058824, 0.30980392]

In [12]:
m = len(y)//2
x_train = x[:m]
y_train = y[:m]
x_test = x[m:m*2]
y_test = y[m:m*2]

x_train.shape , y_train.shape

((13016, 32, 32, 3), (13016,))

In [0]:

#전문가 모델
class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()#상속한 클래스의 생성자 호출 
    self.opt = tf.optimizers.RMSprop(learning_rate=0.01)#Stochatic Gradient Descent 확률적 경사 하강
    self.conv0 = keras.layers.Conv2D(32, [3,3], padding='same', activation=keras.activations.relu)
    self.conv1 = keras.layers.Conv2D(16, [3,3], padding='same', activation=keras.activations.relu)
    self.conv2 = keras.layers.Conv2D(8, [3,3], padding='same', activation=keras.activations.relu)
    self.conv3 = keras.layers.Conv2D(4, [3,3], padding='same', activation=keras.activations.relu)
    self.pool0 = keras.layers.MaxPool2D([2,2], padding='same')
    self.pool1 = keras.layers.MaxPool2D([2,2], padding='same')
    # self.pool2 = keras.layers.MaxPool2D([2,2], padding='same')
    # self.pool3 = keras.layers.MaxPool2D([2,2], padding='same')
    self.flatten = keras.layers.Flatten()
    self.dense1 = keras.layers.Dense(32, activation='sigmoid')
    self.dense0 = keras.layers.Dense(units=10, activation=keras.activations.softmax)
  
  def call(self, x):
    #x (1797, 64)
    x_4d = tf.reshape(x, [-1,32,32,3]) 
    x_4d = tf.cast(x_4d, tf.float32)
    net = self.conv0(x_4d)
    net = self.pool0(net)
    net = self.conv1(net)
    net = self.pool1(net)
    net = self.conv2(net)
    # net = self.pool2(net)
    net = self.conv3(net)
    # net = self.pool3(net)
    net = self.flatten(net)  
    net = self.dense1(net)  
    h = self.dense0(net)
    return h

  def get_loss(self, y, h):
    #학습할때 nan이 발생하는 경우 값을 clip(자르다) (최소값, 최대값) 
    h = tf.clip_by_value(h, 1e-8, 1 - 1e-8) # h 가 0이나 1이 되지 않도록 하는 안전장치 
    cross_entropy = - (y * tf.math.log(h) + (1 - y) * tf.math.log(1 - h)) 
    loss = tf.reduce_mean(cross_entropy)
    return loss

  def get_accuracy(self, y, h):    
    predict = tf.argmax(h, -1)
    
    self.acc = tf.reduce_mean(tf.cast(tf.equal(y, predict), tf.float32)) # True > 1, False > 0 로 cast

  def fit(self, x, y, epoch=1):
    # x : (m, 4), y: (m)    
    y_hot = tf.one_hot(y, depth=10, axis=-1)#(m, 3)  
    for i in range(epoch):
        # iteration=np.ceil(len(y))
        # for j in range(iteration):
        #   start = j*batch_m
        #   end= (j+1)*betch_m
        #   x_batch=x[start:end]
        #   y_batch=y[start:end]
      with tf.GradientTape() as tape: #경사 기록 장치
        h = self.call(x)
        loss = self.get_loss(y_hot, h)        
      grads = tape.gradient(loss, self.trainable_variables) #경사 계산
      self.opt.apply_gradients(zip(grads, self.trainable_variables)) # 가중치에서 경사를 빼기
      self.get_accuracy(y, h)
      
      if i%10==0:
        print('%d/%d loss:%.3f acc:%.3f'%(i, epoch, loss, self.acc))
model = MyModel()

In [0]:
# epoch=100
# batch_m=1000
# for i in range(epoch):
#   iteration=np.ceil(len(y)/batch_m)
#   for j in range(iteration):
#     start = j*batch_m
#     end= (j+1)*batch_m
#     x_batch=x[start:end]
#     y_batch=y[start:end]

In [26]:
model.fit(x_train, y_train, 800)

0/800 loss:0.367 acc:0.077
10/800 loss:0.316 acc:0.195
20/800 loss:0.317 acc:0.161
30/800 loss:0.316 acc:0.163
40/800 loss:0.306 acc:0.262
50/800 loss:0.289 acc:0.318
60/800 loss:0.266 acc:0.379
70/800 loss:0.258 acc:0.434
80/800 loss:0.233 acc:0.497
90/800 loss:0.217 acc:0.530
100/800 loss:0.191 acc:0.612
110/800 loss:0.187 acc:0.618
120/800 loss:0.180 acc:0.639
130/800 loss:0.177 acc:0.642
140/800 loss:0.154 acc:0.698
150/800 loss:0.145 acc:0.718
160/800 loss:0.149 acc:0.703
170/800 loss:0.146 acc:0.708
180/800 loss:0.136 acc:0.733
190/800 loss:0.130 acc:0.745
200/800 loss:0.146 acc:0.700
210/800 loss:0.116 acc:0.777
220/800 loss:0.125 acc:0.753
230/800 loss:0.125 acc:0.754
240/800 loss:0.108 acc:0.790
250/800 loss:0.106 acc:0.796
260/800 loss:0.125 acc:0.749
270/800 loss:0.102 acc:0.806
280/800 loss:0.098 acc:0.813
290/800 loss:0.096 acc:0.815
300/800 loss:0.098 acc:0.811
310/800 loss:0.102 acc:0.802
320/800 loss:0.094 acc:0.821
330/800 loss:0.085 acc:0.844
340/800 loss:0.085 acc:0.